In [1]:
import pandas as pd
import requests


In [ ]:
limit = 1000
def records_for_role(role_id):
    records = []
    url = f"https://directory.spineservices.nhs.uk/ORD/2-0-0/organisations?PrimaryRoleId={role_id}&Limit={limit}"
    rsp = requests.get(url)
    new_records = rsp.json()["Organisations"]
    while new_records:
        records.extend(new_records)
        print(len(records))
        offset = str(len(records))
        url = f"https://directory.spineservices.nhs.uk/ORD/2-0-0/organisations?PrimaryRoleId={role_id}&Limit={limit}&Offset={offset}"
        rsp = requests.get(url)
        new_records = rsp.json()["Organisations"]
    return pd.DataFrame.from_records(records)
role_ids = ["RO141", "RO172", "RO98","RO197","RO107"]
dfs = [records_for_role(role_id) for role_id in role_ids]
df = pd.concat(dfs)

In [21]:
df=df.reset_index()

In [11]:
#Calculate overall cost for warfarin tablets for 3 months to end of Feb 2020 for TPP practices
sql='''
SELECT
  pct, practice, bnf_code, bnf_name, quantity_per_item,
  SUM(CASE WHEN month = '2020-01-01' THEN items ELSE 0 END) AS jan_2020_items,
  SUM(CASE WHEN month = '2021-01-01' THEN items ELSE 0 END) AS jan_2021_items
FROM
  ebmdatalab.hscic.raw_prescribing_normalised AS rx
WHERE
  month IN ('2021-01-01','2020-01-01')
  AND bnf_code LIKE '041003%'
GROUP BY
  pct, practice, bnf_code, bnf_name, quantity_per_item
'''
exportfile = os.path.join("..","data","adp_df.csv") #set path for data cache
oat_df = bq.cached_read(sql, csv_path=exportfile, use_cache=True) #save dataframe to csv
display(oat_df) #show dataframe as a table

Downloading: 100%|██████████| 67870/67870 [00:03<00:00, 21553.99rows/s]


,pct,practice,bnf_code,bnf_name,quantity_per_item,jan_2020_items,jan_2021_items
0,15F,B86033,0410030E0AAANAN,Naltrexone 1.5mg capsules,56.0,3,0
1,72Q,G85633,0410030B0AAAAAA,Buprenorphine 8mg / Naloxone 2mg sublingual ta...,21.0,1,0
2,15N,L83112,0410030A0BCABAD,Prefibin 2mg sublingual tablets,42.0,2,0
3,116,Y00107,0410030B0BBABAB,Suboxone 2mg/500microgram sublingual tablets,28.0,2,2
4,09A,E87011,0410030C0AABABA,Methadone 10mg/ml oral solution sugar free,266.0,2,0
...,...,...,...,...,...,...,...
67865,NMS,Y06021,0410030E0AAAAAA,Naltrexone 50mg tablets,9.0,4,0
67866,36L,H85003,0410030E0AAAAAA,Naltrexone 50mg tablets,56.0,0,1
67867,NKI,Y06027,0410030E0AAAAAA,Naltrexone 50mg tablets,14.0,0,2
67868,00T,P82034,0410030E0AAAAAA,Naltrexone 50mg tablets,28.0,0,1


In [33]:
test = pd.merge(df, oat_df, how="right", on=["Name", "pct"])

KeyError: 'Name'

In [30]:
oat_df.dtypes

pct                   object
practice              object
bnf_code              object
bnf_name              object
quantity_per_item    float64
jan_2020_items         int64
jan_2021_items         int64
dtype: object

In [32]:
oat_df.reset_index()

,index,pct,practice,bnf_code,bnf_name,quantity_per_item,jan_2020_items,jan_2021_items
0,0,15F,B86033,0410030E0AAANAN,Naltrexone 1.5mg capsules,56.0,3,0
1,1,72Q,G85633,0410030B0AAAAAA,Buprenorphine 8mg / Naloxone 2mg sublingual ta...,21.0,1,0
2,2,15N,L83112,0410030A0BCABAD,Prefibin 2mg sublingual tablets,42.0,2,0
3,3,116,Y00107,0410030B0BBABAB,Suboxone 2mg/500microgram sublingual tablets,28.0,2,2
4,4,09A,E87011,0410030C0AABABA,Methadone 10mg/ml oral solution sugar free,266.0,2,0
...,...,...,...,...,...,...,...,...
67865,67865,NMS,Y06021,0410030E0AAAAAA,Naltrexone 50mg tablets,9.0,4,0
67866,67866,36L,H85003,0410030E0AAAAAA,Naltrexone 50mg tablets,56.0,0,1
67867,67867,NKI,Y06027,0410030E0AAAAAA,Naltrexone 50mg tablets,14.0,0,2
67868,67868,00T,P82034,0410030E0AAAAAA,Naltrexone 50mg tablets,28.0,0,1


In [34]:
from ebmdatalab import bq
limit = 1000
def records_for_role(role_id):
    records = []
    url = f"https://directory.spineservices.nhs.uk/ORD/2-0-0/organisations?PrimaryRoleId={role_id}&Limit={limit}"
    rsp = requests.get(url)
    new_records = rsp.json()["Organisations"]
    while new_records:
        records.extend(new_records)
        print(len(records))
        offset = str(len(records))
        url = f"https://directory.spineservices.nhs.uk/ORD/2-0-0/organisations?PrimaryRoleId={role_id}&Limit={limit}&Offset={offset}"
        rsp = requests.get(url)
        new_records = rsp.json()["Organisations"]
    return pd.DataFrame.from_records(records)
role_ids = ["RO99", "RO198", "RO177"]
dfs_prac = [records_for_role(role_id) for role_id in role_ids]
df_prac = pd.concat(dfs_prac)

1000
2000
2167
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
30183
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
17419


In [35]:
df_prac.head()

,Name,OrgId,Status,OrgRecordClass,PostCode,LastChangeDate,PrimaryRoleId,PrimaryRoleDescription,OrgLink
0,ADUR DISTRICT COUNCIL,105,Active,RC1,BN11 1HS,2020-04-05,RO141,LOCAL AUTHORITY,https://directory.spineservices.nhs.uk/ORD/2-0...
1,ALLERDALE BOROUGH COUNCIL,115,Active,RC1,CA14 3YJ,2020-04-03,RO141,LOCAL AUTHORITY,https://directory.spineservices.nhs.uk/ORD/2-0...
2,AMBER VALLEY BOROUGH COUNCIL,118,Active,RC1,DE5 3BT,2020-04-05,RO141,LOCAL AUTHORITY,https://directory.spineservices.nhs.uk/ORD/2-0...
3,ARUN DISTRICT COUNCIL,119,Active,RC1,BN17 5LF,2020-04-05,RO141,LOCAL AUTHORITY,https://directory.spineservices.nhs.uk/ORD/2-0...
4,ASHFIELD DISTRICT COUNCIL,120,Active,RC1,NG17 8DA,2020-04-05,RO141,LOCAL AUTHORITY,https://directory.spineservices.nhs.uk/ORD/2-0...


In [36]:
df_prac = pd.concat(dfs_prac)

In [37]:
df_prac.head()

,Name,OrgId,Status,OrgRecordClass,PostCode,LastChangeDate,PrimaryRoleId,PrimaryRoleDescription,OrgLink
0,111 CALL CENTRE,06HAK,Active,RC2,PE3 6DB,2013-05-23,RO99,CLINICAL COMMISSIONING GROUP SITE,https://directory.spineservices.nhs.uk/ORD/2-0...
1,1829 BUILDING (CCG HQ),02FAA,Inactive,RC2,CH2 1UL,2021-04-26,RO99,CLINICAL COMMISSIONING GROUP SITE,https://directory.spineservices.nhs.uk/ORD/2-0...
2,1829 BUILDING - CCG SATELLITE SITE,27DAC,Active,RC2,CH2 1HJ,2020-03-05,RO99,CLINICAL COMMISSIONING GROUP SITE,https://directory.spineservices.nhs.uk/ORD/2-0...
3,5 FOUNTAIN COURT,12AAN,Active,RC2,BS32 4LA,2018-04-19,RO99,CLINICAL COMMISSIONING GROUP SITE,https://directory.spineservices.nhs.uk/ORD/2-0...
4,A101CODY TECHNOLOGY PARK,08NTP,Active,RC2,GU14 0LZ,2021-04-06,RO99,CLINICAL COMMISSIONING GROUP SITE,https://directory.spineservices.nhs.uk/ORD/2-0...


In [56]:
oat_prac = oat_df.groupby(['practice'],as_index=False)

In [69]:
oat_df_small = oat_df['practice'].to_frame()

In [70]:
oat_df = oat_df_small.groupby(['practice'])

In [73]:
oat_df_small.head(5000)

,practice
0,B86033
1,G85633
2,L83112
3,Y00107
4,E87011
...,...
4995,Y05875
4996,G85716
4997,E87009
4998,L81054


In [75]:
#Calculate overall cost for warfarin tablets for 3 months to end of Feb 2020 for TPP practices
sql='''
SELECT
practice
FROM
  ebmdatalab.hscic.raw_prescribing_normalised AS rx
WHERE
  month IN ('2021-01-01','2020-01-01')
  AND bnf_code LIKE '041003%'
GROUP BY
 practice
'''
exportfile = os.path.join("..","data","oat_prac_df.csv") #set path for data cache
oat_prac_df = bq.cached_read(sql, csv_path=exportfile, use_cache=True) #save dataframe to csv
display(oat_prac_df) #show dataframe as a table

Downloading: 100%|██████████| 2590/2590 [00:00<00:00, 14964.59rows/s]


,practice
0,M83052
1,Y02669
2,L85015
3,N82097
4,Y02077
...,...
2585,G82044
2586,P84035
2587,M85679
2588,A84008


In [91]:
for index, row in oat_prac_df:
    print(row)

ValueError: too many values to unpack (expected 2)

In [89]:
i = 0
#new_df = pandas.DataFrame(columns = ['a','b','c','d'])
for index,row in oat_prac_df.iterrows():
    url = 'https://directory.spineservices.nhs.uk/ORD/2-0-0/organisations/'
    d = '{ + row['practice'] + '}'
   # j = json.loads(d)
   # response = requests.post(url,json = j)

   # data = response.json()
   # for new_data in data['c']:
      #  new_df.loc[i] = [row['a'],row['b'],row['c'],new_data]
        i += 1

SyntaxError: invalid syntax (<ipython-input-89-4497e597ecac>, line 5)

In [108]:
def build_request(row):
    return row['practice']}

request_bodies = oat_prac_df.apply(build_request, axis=1).tolist()

In [124]:
    api_results = []
    for x in request_bodies:
        url ='https://directory.spineservices.nhs.uk/ORD/2-0-0/organisations/' + x
        request = requests.get(url)
        api_results.append(request.json())

In [111]:
url = 'https://directory.spineservices.nhs.uk/ORD/2-0-0/organisations/M91013'

In [ ]:
r = requests.get(url)



    records = []
    url = f"https://directory.spineservices.nhs.uk/ORD/2-0-0/organisations/M91013?format=xml"
    rsp = requests.get(url)
    new_records = pd.read_xml(rsp)

In [115]:
json

{'Organisation': {'Name': 'LOCKFIELD SURGERY',
  'Date': [{'Type': 'Operational', 'Start': '1974-04-01'}],
  'OrgId': {'root': '2.16.840.1.113883.2.1.3.2.4.18.48',
   'assigningAuthorityName': 'HSCIC',
   'extension': 'M91013'},
  'Status': 'Active',
  'LastChangeDate': '2021-04-01',
  'orgRecordClass': 'RC1',
  'GeoLoc': {'Location': {'AddrLn1': 'WILLENHALL HEALTH CENTRE',
    'AddrLn2': 'CROFT STREET',
    'Town': 'WILLENHALL',
    'PostCode': 'WV13 2DR',
    'Country': 'ENGLAND',
    'UPRN': 10090065767}},
  'Contacts': {'Contact': [{'type': 'tel', 'value': '01902 639000'}]},
  'Roles': {'Role': [{'id': 'RO177',
     'uniqueRoleId': 111870,
     'primaryRole': True,
     'Date': [{'Type': 'Operational', 'Start': '1974-04-01'}],
     'Status': 'Active'},
    {'id': 'RO76',
     'uniqueRoleId': 187922,
     'Date': [{'Type': 'Operational', 'Start': '2014-04-15'}],
     'Status': 'Active'}]},
  'Rels': {'Rel': [{'Date': [{'Type': 'Operational', 'Start': '2021-04-01'}],
     'Status': '

In [118]:
new_records

{'Name': 'LOCKFIELD SURGERY',
 'Date': [{'Type': 'Operational', 'Start': '1974-04-01'}],
 'OrgId': {'root': '2.16.840.1.113883.2.1.3.2.4.18.48',
  'assigningAuthorityName': 'HSCIC',
  'extension': 'M91013'},
 'Status': 'Active',
 'LastChangeDate': '2021-04-01',
 'orgRecordClass': 'RC1',
 'GeoLoc': {'Location': {'AddrLn1': 'WILLENHALL HEALTH CENTRE',
   'AddrLn2': 'CROFT STREET',
   'Town': 'WILLENHALL',
   'PostCode': 'WV13 2DR',
   'Country': 'ENGLAND',
   'UPRN': 10090065767}},
 'Contacts': {'Contact': [{'type': 'tel', 'value': '01902 639000'}]},
 'Roles': {'Role': [{'id': 'RO177',
    'uniqueRoleId': 111870,
    'primaryRole': True,
    'Date': [{'Type': 'Operational', 'Start': '1974-04-01'}],
    'Status': 'Active'},
   {'id': 'RO76',
    'uniqueRoleId': 187922,
    'Date': [{'Type': 'Operational', 'Start': '2014-04-15'}],
    'Status': 'Active'}]},
 'Rels': {'Rel': [{'Date': [{'Type': 'Operational', 'Start': '2021-04-01'}],
    'Status': 'Active',
    'Target': {'OrgId': {'root': 

In [125]:
api_results

[{'Organisation': {'Name': 'WEEPING CROSS HEALTH CENTRE',
   'Date': [{'Type': 'Operational', 'Start': '1974-04-01'}],
   'OrgId': {'root': '2.16.840.1.113883.2.1.3.2.4.18.48',
    'assigningAuthorityName': 'HSCIC',
    'extension': 'M83052'},
   'Status': 'Active',
   'LastChangeDate': '2021-02-04',
   'orgRecordClass': 'RC1',
   'GeoLoc': {'Location': {'AddrLn1': 'BODMIN AVENUE',
     'AddrLn2': 'WEEPING CROSS',
     'Town': 'STAFFORD',
     'County': 'STAFFORDSHIRE',
     'PostCode': 'ST17 0EG',
     'Country': 'ENGLAND',
     'UPRN': 200001321618}},
   'Contacts': {'Contact': [{'type': 'tel', 'value': '01785 662505'}]},
   'Roles': {'Role': [{'id': 'RO177',
      'uniqueRoleId': 75665,
      'primaryRole': True,
      'Date': [{'Type': 'Operational', 'Start': '1974-04-01'}],
      'Status': 'Active'},
     {'id': 'RO76',
      'uniqueRoleId': 187177,
      'Date': [{'Type': 'Operational', 'Start': '2014-04-15'}],
      'Status': 'Active'}]},
   'Rels': {'Rel': [{'Date': [{'Type': '

In [126]:
dataframe = pd.DataFrame.from_dict(api_results, orient="index")

AttributeError: 'list' object has no attribute 'values'

In [132]:
dfdf = pd.read_json(api_results, orient='records')

ValueError: Invalid file path or buffer object type: <class 'list'>

In [2]:
print(pd.__version__)

1.2.4
